# CA4 - Question 1
### We needed strict parenthesis and date checks so I add non-regex validators for them.

In [69]:
import re

class ValidatorMeta(type):
    def __new__(cls, name, bases, dct):
        
        validation_rules = {
            'email': r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9.-]+\.(com|org)$',
            'phone_number': r'^\+98\d{8}$',
            'password': r'^(?=.*[A-Z])(?=.*[a-z])(?=.*\d)(?=.*[*@%!]).{8,12}$',
            'product_code': r'^[A-Z]{2}\d{2,4}[a-z]?(?:-?v\d{1,2})?$',
            'stop_word': r'^\s*(stop|Stop)\s*$',
            'repeated_phrase': r'.*\b(some students|many employees)\b.*\b\1\b',
            'quotation': r"^[\"'][a-zA-Z0-9 ]+[\"']$",
        }

        custom_error_messages = {
            'email': 'Email must have a valid format and end with .com or .org.',
            'phone_number': 'Phone number must start with +98 and be followed by 8 digits.',
            'password': 'Password must be between 8 and 12 characters and include at least one uppercase letter, one lowercase letter, one number, and one special character (*, @, %, !).',
            'product_code': 'Product code must consist of 2 uppercase letters, 2 to 4 digits, an optional lowercase letter, and an optional version number (v1-v99).',
            'stop_word': 'The word "Stop" or "stop" must be separate and not part of a larger word or attached to punctuation.',
            'repeated_phrase': 'The phrase "some students" or "many employees" must repeat exactly, with no different terms in between.',
            'date': 'Date must be in YYYY/MM/DD format, with valid month and day values.',
            'quotation': 'Text must be enclosed in balanced single or double quotes and contain only letters, numbers, and spaces.',
            'parenthesis': 'Parentheses must be balanced with no unmatched opening or closing parentheses.',
        }

        # Method generator for validation functions
        def create_validator(field, rule, custom_message):
            def validator(self, value):
                if not re.match(rule, value):
                    raise ValueError(custom_message)
                return True
            return validator

        # Add validation methods to the class dynamically
        for field, rule in validation_rules.items():
            dct[f'validate_{field}'] = create_validator(field, rule, custom_error_messages.get(field, f"Invalid {field}"))

        # Stack-based parenthesis balancer
        def validate_parenthesis(self, value):
            stack = []
            for char in value:
                if char == '(':
                    stack.append(char)
                elif char == ')':
                    if not stack:
                        raise ValueError(custom_error_messages['parenthesis'])
                    stack.pop()
            if stack:
                raise ValueError(custom_error_messages['parenthesis'])
            return True

        dct['validate_parenthesis'] = validate_parenthesis

        # Custom date validator
        def validate_date(self, value):
            import datetime
            try:
                datetime.datetime.strptime(value, '%Y/%m/%d')
                return True
            except ValueError:
                raise ValueError(custom_error_messages['date'])

        dct['validate_date'] = validate_date

        return super().__new__(cls, name, bases, dct)


class FormValidator(metaclass=ValidatorMeta):
    """Concrete validator class with all validation methods"""
    pass

In [70]:
validator = FormValidator()

def run_all_tests():
    print("=== EMAIL TESTS ===")
    valid_emails = ["user@example.com", "john.doe@mail.org", "noor@ut.ac.com", "example@example.com"]
    invalid_emails = ["example.com", "user@example.net", "invalid@domain", "user@example.comm", "noor@ut.ac.ir"]

    for email in valid_emails:
        try:
            validator.validate_email(email)
            print(f"✅ '{email}' is valid")
        except ValueError as e:
            print(f"❌ '{email}' should be valid: {e}")

    for email in invalid_emails:
        try:
            validator.validate_email(email)
            print(f"❌ '{email}' should be invalid")
        except ValueError as e:
            print(f"✅ '{email}' correctly rejected: {e}")

    print("\n=== PHONE NUMBER TESTS ===")
    valid_phones = ["+9812345678", "+9800000000"]
    invalid_phones = ["+9912345678", "+981234567", "+98123456789", "9812345678", "0912345678"]

    for phone in valid_phones:
        try:
            validator.validate_phone_number(phone)
            print(f"✅ '{phone}' is valid")
        except ValueError as e:
            print(f"❌ '{phone}' should be valid: {e}")

    for phone in invalid_phones:
        try:
            validator.validate_phone_number(phone)
            print(f"❌ '{phone}' should be invalid")
        except ValueError as e:
            print(f"✅ '{phone}' correctly rejected: {e}")

    print("\n=== PASSWORD TESTS ===")
    valid_passwords = [
        "Pass*word1",
        "passWord1!",
        "SecureP@ss99"
    ]
    invalid_passwords = [   
        "PASSWORD1!",   
        "password1!",  
        "PassworD!",
        "Pass!*",
        "Passworwibuteo*&^#%"
    ]

    for pwd in valid_passwords:
        try:
            validator.validate_password(pwd)
            print(f"✅ '{pwd}' is valid")
        except ValueError as e:
            print(f"❌ '{pwd}' should be valid: {e}")

    for pwd in invalid_passwords:
        try:
            validator.validate_password(pwd)
            print(f"❌ '{pwd}' should be invalid")
        except ValueError as e:
            print(f"✅ '{pwd}' correctly rejected: {e}")

    print("\n=== PRODUCT CODE TESTS ===")
    valid_product_codes = ["AB1234-v1", "CD12-v34"]
    invalid_product_codes = ["A123B", "abcd12", "AB12345", "AB12-av100", "AB12-avv99", "EF123-bv99"]

    for code in valid_product_codes:
        try:
            validator.validate_product_code(code)
            print(f"✅ '{code}' is valid")
        except ValueError as e:
            print(f"❌ '{code}' should be valid: {e}")

    for code in invalid_product_codes:
        try:
            validator.validate_product_code(code)
            print(f"❌ '{code}' should be invalid")
        except ValueError as e:
            print(f"✅ '{code}' correctly rejected: {e}")

    print("\n=== STOP WORD TESTS ===")
    valid_stop_words = ["Stop", "stop ", "stop", " stop", " Stop", "Stop   "]
    invalid_stop_words = ["Stop!", "Stop.", " StOp ", "Stopping", "startstop", "stopword", "xstop", "stops", "Stopped"]

    for word in valid_stop_words:
        try:
            validator.validate_stop_word(word)
            print(f"✅ '{word}' is valid")
        except ValueError as e:
            print(f"❌ '{word}' should be valid: {e}")

    for word in invalid_stop_words:
        try:
            validator.validate_stop_word(word)
            print(f"❌ '{word}' should be invalid")
        except ValueError as e:
            print(f"✅ '{word}' correctly rejected: {e}")

    print("\n=== REPEATED PHRASE TESTS ===")
    valid_phrases = ["some students like some students", "many employees work with many employees"]
    invalid_phrases = ["some students collaborate with many employees", "some student repeats some students"]

    for phrase in valid_phrases:
        try:
            validator.validate_repeated_phrase(phrase)
            print(f"✅ '{phrase}' is valid")
        except ValueError as e:
            print(f"❌ '{phrase}' should be valid: {e}")

    for phrase in invalid_phrases:
        try:
            validator.validate_repeated_phrase(phrase)
            print(f"❌ '{phrase}' should be invalid")
        except ValueError as e:
            print(f"✅ '{phrase}' correctly rejected: {e}")

    print("\n=== DATE TESTS ===")
    valid_dates = ["1404/03/20", "2023/12/31", "1999/01/01"]
    invalid_dates = ["1404/13/32", "2023/00/15", "2023/13/01", "2023/04/31"]

    for date in valid_dates:
        try:
            validator.validate_date(date)
            print(f"✅ '{date}' is valid")
        except ValueError as e:
            print(f"❌ '{date}' should be valid: {e}")

    for date in invalid_dates:
        try:
            validator.validate_date(date)
            print(f"❌ '{date}' should be invalid")
        except ValueError as e:
            print(f"✅ '{date}' correctly rejected: {e}")

    print("\n=== QUOTATION TESTS ===")
    valid_quotes = ['"This is valid"', "'Single quote'", '"Only letters numbers 123"']
    invalid_quotes = ['"No closing', "'Unclosed", '"Invalid inside#"', 'No quotes']

    for quote in valid_quotes:
        try:
            validator.validate_quotation(quote)
            print(f"✅ '{quote}' is valid")
        except ValueError as e:
            print(f"❌ '{quote}' should be valid: {e}")

    for quote in invalid_quotes:
        try:
            validator.validate_quotation(quote)
            print(f"❌ '{quote}' should be invalid")
        except ValueError as e:
            print(f"✅ '{quote}' correctly rejected: {e}")

    print("\n=== PARENTHESIS TESTS ===")
    valid_parens = [
        "No parentheses here",
        "This is a valid parentheses",
        "(balanced)",
        "((nested))",
        "This (is) valid",
        "Multiple ()() pairs"
    ]
    invalid_parens = [
        "(",
        ")",
        "(()",
        "())",
        "Mismatched) (wrong order",
        "Extra ) at end",
    ]

    for text in valid_parens:
        try:
            validator.validate_parenthesis(text)
            print(f"✅ '{text}' is valid")
        except ValueError as e:
            print(f"❌ '{text}' should be valid: {e}")

    for text in invalid_parens:
        try:
            validator.validate_parenthesis(text)
            print(f"❌ '{text}' should be invalid")
        except ValueError as e:
            print(f"✅ '{text}' correctly rejected: {e}")

run_all_tests()

=== EMAIL TESTS ===
✅ 'user@example.com' is valid
✅ 'john.doe@mail.org' is valid
✅ 'noor@ut.ac.com' is valid
✅ 'example@example.com' is valid
✅ 'example.com' correctly rejected: Email must have a valid format and end with .com or .org.
✅ 'user@example.net' correctly rejected: Email must have a valid format and end with .com or .org.
✅ 'invalid@domain' correctly rejected: Email must have a valid format and end with .com or .org.
✅ 'user@example.comm' correctly rejected: Email must have a valid format and end with .com or .org.
✅ 'noor@ut.ac.ir' correctly rejected: Email must have a valid format and end with .com or .org.

=== PHONE NUMBER TESTS ===
✅ '+9812345678' is valid
✅ '+9800000000' is valid
✅ '+9912345678' correctly rejected: Phone number must start with +98 and be followed by 8 digits.
✅ '+981234567' correctly rejected: Phone number must start with +98 and be followed by 8 digits.
✅ '+98123456789' correctly rejected: Phone number must start with +98 and be followed by 8 digits.
